# 1 코드 실행환경

Google Colab

런타임 유형 : None

Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic

Ubuntu 18.04.6 LTS

Python 3.7.13

In [ ]:
import platform
platform.platform()

'Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic'

In [ ]:
!cat /etc/issue.net

Ubuntu 18.04.6 LTS


In [ ]:
!python --version

Python 3.7.13


## 1.1 데이터 입/출력 경로 지정

구글 코랩 사용시 구글 드라이브 연결 사용

로컬 환경 사용시 로컬 환경 경로로 설정

In [2]:
#구글 드라이브 연결
from google.colab import drive

drive.mount('/content/gdrive/')

#코랩 환경 경로 설정 
DATA_PATH = '/content/gdrive/MyDrive/dacon-235927-kops/data/'

Mounted at /content/gdrive/


In [ ]:
# 로컬 환경 경로 설정
DATA_PATH = '/data/'

## 1.2 필요 라이브러리 설치

In [3]:
# optuna 설치
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 59.5 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 
     |████████████████████████████████| 112 kB 81.9 MB/s 
     |████████████████████████████████| 147 kB 74.0 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 


## 1.3 라이브러리 로드

In [4]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, train_test_split

import lightgbm as lgb
from lightgbm import LGBMRegressor

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.decomposition import PCA
import joblib

import warnings

warnings.filterwarnings("ignore")

## 1.4 시드 고정

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) 
SEED = 42

# 2 데이터 전처리

In [6]:
# 데이터 로드
train = pd.read_csv(DATA_PATH + 'train.csv')

# X Y 데이터 분리
X_train = train.filter(regex='X') # Input : X Featrue
Y_train = train.filter(regex='Y') # Output : Y Feature

In [7]:
# 결과에 영향 낮은 인자 제거
X_train = X_train.drop(['X_04', 'X_23', 'X_47', 'X_48', 'X_10', 'X_11', 'X_02'], axis=1)

In [8]:
# X_33 이상치 제거
drop_idx = X_train.loc[X_train['X_33'] > 6 ].index

X_train = X_train.drop(drop_idx, axis = 0)
Y_train = Y_train.drop(drop_idx, axis = 0)

X_train = X_train.reset_index(drop = True)
Y_train = Y_train.reset_index(drop = True)

# 3 평가산식 정의

In [9]:
def nrmse(y_val, y_pred):
  rmse = mean_squared_error(y_val, y_pred, squared=False)
  nrmse = rmse/np.mean(np.abs(y_val))
  return nrmse

def lg_nrmse(y_val, y_pred):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여

    y_val = pd.DataFrame(y_val)
    y_pred = pd.DataFrame(y_pred)

    all_nrmse = []
    for idx in range(0,14):
        all_nrmse.append(nrmse(y_val.iloc[:,idx], y_pred.iloc[:,idx]))
        
    score = 1.2 * np.sum(all_nrmse[:7]) + 1.0 * np.sum(all_nrmse[7:14])
    return score

# 4 PCA 최적화 준비

In [10]:
# PCA 클래스 설정
class PCA_transform:

  def __init__(self):
    self.cols_list = []
    self.pca_list = []
    self.n_pca_list = []
    self.size = 0
  
  # PCA 클래스의 학습 및 input 값 변환
  def fit_transform(self, X_input, col, n_pca):
    
    if (n_pca == 0):
      return X_input

    X_pca = X_input[col]

    # n차원으로 차원 축소, target 정보는 제외
    pca = PCA(n_components = n_pca)

    # PCA 학습
    pca.fit(X_pca)

    # PCA transform 후 데이터프레임으로 자료형 변경
    X_pca = pca.transform(X_pca)
    X_pca = pd.DataFrame(X_pca, columns = self.naming(n_pca))

    X_input = pd.concat([X_input, X_pca], axis = 1)
    X_input = X_input.drop(col, axis = 1)

    self.cols_list.append(col)
    self.pca_list.append(pca)
    self.n_pca_list.append(n_pca)
    self.size += 1

    return X_input

  # 학습된 PCA 값으로 transform
  def transform(self, X_input):
    for idx in range(self.size):
      X_input = self._idx_transform(X_input, idx)
    
    return X_input

  # n번째 PCA 변환
  def _idx_transform(self, X_input, idx):
    X_pca = X_input[self.cols_list[idx]]

    # pca transform 후 데이터프레임으로 자료형 변경
    X_pca = self.pca_list[idx].transform(X_pca)
    X_pca = pd.DataFrame(X_pca, columns = self.naming(self.n_pca_list[idx], idx))

    X_input = pd.concat([X_input, X_pca], axis = 1)
    X_input = X_input.drop(self.cols_list[idx], axis = 1)

    return X_input

  # PCA 된 컬럼 이름 규칙
  def naming(self, number, name = None):
    if (name is None):
      name = self.size
    names = []
    for idx in range(number):
      names.append(f'PCA_{str(name)}_{idx}')
    return names

In [11]:
# 교차 검증을 통한 모델 학습
def train_model(model, X_input, Y_input, n_folds):
  # 결과를 넣을 데이터 프레임 생성
  train_fold_preds = pd.DataFrame()
  
  X_input = X_input.to_numpy()

  kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)

  # Y 각 14개 인자에 대해 각각 학습 후 예측
  for idx in range(0, 14):
    # 예측 결과를 넣을 ndarray 생성
    train_fold_pred = np.zeros((X_input.shape[0] ,1 ))
    y_input = Y_input.iloc[:,idx]

    # n_folds 만큼 KFold 학습
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_input)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        X_tr = X_input[train_index]
        y_tr = y_input[train_index]
        X_val = X_input[valid_index]

        #폴드 세트 내부에서 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)  
        #폴드 세트 내부에서 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_val).reshape(-1,1)
    
    # 예측된 결과를 데이터 프레임에 저장
    train_fold_preds = pd.concat([train_fold_preds, pd.DataFrame(train_fold_pred)],axis=1)

  return train_fold_preds

In [16]:
# Oputna로 탐색할 목적 함수 생성
def pca_objective(trial: Trial, X_input, Y_input, model, n_folds):
  # Optuna로 탐색할 PCA 범위 설정
  pca_1 = trial.suggest_int("pca_1", 0, 2)
  pca_2 = trial.suggest_int("pca_2", 0, 2)
  pca_3 = trial.suggest_int("pca_3", 0, 5)
  pca_4 = trial.suggest_int("pca_4", 0, 3)
  pca_5 = trial.suggest_int("pca_5", 0, 5)
  pca_6 = trial.suggest_int("pca_6", 0, 3)
  pca_7 = trial.suggest_int("pca_7", 0, 3)
  pca_8 = trial.suggest_int("pca_8", 0, 2)
  pca_9 = trial.suggest_int("pca_9", 0, 4)
  pca_10 = trial.suggest_int("pca_10", 0, 6)

  # 임의로 탐색된 변수로 X_input값을 PCA 수행
  pca = PCA_transform()
  X_input = pca.fit_transform(X_input, ['X_01', 'X_05', 'X_06'], pca_1)
  X_input = pca.fit_transform(X_input, ['X_07', 'X_08', 'X_09'], pca_2)
  X_input = pca.fit_transform(X_input, ['X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18'], pca_3)
  X_input = pca.fit_transform(X_input, ['X_19', 'X_20', 'X_21', 'X_22'], pca_4)
  X_input = pca.fit_transform(X_input, ['X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29'], pca_5)
  X_input = pca.fit_transform(X_input, ['X_30', 'X_31', 'X_32', 'X_33'], pca_6)
  X_input = pca.fit_transform(X_input, ['X_34', 'X_35', 'X_36', 'X_37'], pca_7)
  X_input = pca.fit_transform(X_input, ['X_38', 'X_39', 'X_40'], pca_8)
  X_input = pca.fit_transform(X_input, ['X_41', 'X_42', 'X_43', 'X_44', 'X_45'], pca_9)
  X_input = pca.fit_transform(X_input, ['X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56'], pca_10)

  # PCA 변환된 X_input 값으로 교차검증을 통한 예측값 생성
  Y_pred = train_model(model, X_input, Y_input, n_folds)

  # 평가산식으로 score 계산
  score = lg_nrmse(Y_input, Y_pred)

  return score

In [17]:
# X_input, Y_input : 학습 데이터
# model : 사용할 ML 모델
# n_folds : fold 횟수
# n_iter : 시도 횟수
def start_study(X_input, Y_input, model, n_folds, n_iter):
  try:
    # 학습된 데이터가 있을 시 이어서 학습
    study = joblib.load(DATA_PATH + "PCA_study.pkl")
    print('Study loaded')
  except:
    # 학습된 데이터가 없을 시 새로운 학습 데이터 생성
    # 학습방향 : 최소화
    study = optuna.create_study(direction='minimize')
  
  while (n_iter > len(study.trials)):
    # 학습 시작
    study.optimize(lambda trial : pca_objective(trial, X_input,  Y_input, model, n_folds), n_trials=1)

    # 결과 저장
    joblib.dump(study, DATA_PATH + "PCA_study.pkl")

  # 점수 및 최적화된 인자 출력
  print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

# 5 PCA 최적화 탐색

실행시마다 최적화 된 값 변화

최적화된 PCA 값

value: 1.9487536436470618

parameters: {'pca_1': 0, 'pca_2': 0, 'pca_3': 5, 'pca_4': 2, 'pca_5': 0, 'pca_6': 0, 'pca_7': 1, 'pca_8': 0, 'pca_9': 1, 'pca_10': 2}

In [18]:
# 학습 모델로 속도와 성능이 좋은 LGBM 선정
lgb_model = LGBMRegressor(verbose = -1, random_state=SEED)

In [ ]:
# Fold 10으로 학습
start_study(X_train, Y_train, lgb_model, 10, 100)

In [20]:
study = joblib.load(DATA_PATH + "PCA_study.pkl")

In [21]:
study.best_params

{'pca_1': 0,
 'pca_2': 0,
 'pca_3': 5,
 'pca_4': 2,
 'pca_5': 0,
 'pca_6': 0,
 'pca_7': 1,
 'pca_8': 0,
 'pca_9': 1,
 'pca_10': 2}